In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PanTadeusz").getOrCreate()

rdd = spark.sparkContext.textFile("pan-tadeusz.txt")

tadeusz_lines = rdd.filter(lambda line: "Tadeusz" in line)

line_count = tadeusz_lines.count()

print(f"Liczba linii zawierających słowo 'Tadeusz': {line_count}")

Liczba linii zawierających słowo 'Tadeusz': 182


In [7]:
sorted_lines = rdd.map(lambda line: (len(line), line)).sortByKey(ascending=False)

top_3_longest_lines = sorted_lines.take(3)

print("Trzy najdłuższe linie:")
for length, line in top_3_longest_lines:
    print(f"{length}: {line}")

Trzy najdłuższe linie:
536: O niebezpieczeństwach wynikających z nieporządnego obozowania — Odsiecz niespodziana — Smutne położenie szlachty — Odwiedziny kwestarskie są wróżbą ratunku — Major Płut zbytnią zalotnością ściąga na siebie burzę — Wystrzał z krócicy, hasło boju — Czyny Kropiciela, czyny i niebezpieczeństwa Maćka — Konewka zasadzką ocala Soplicowo — Posiłki jezdne, atak na piechotę — Czyny Tadeusza — Pojedynek dowódców przerwany zdradą — Wojski stanowczym manewrem przechyla szalę boju — Czyny krwawe Gerwazego — Podkomorzy zwycięzca wspaniałomyślny.
452: Plany myśliwskie Telimeny — Ogrodniczka wybiera się na wielki świat i słucha nauk opiekunki — Strzelcy wracają — Wielkie zadziwienie Tadeusza — Spotkanie się powtórne w Świątyni dumania i zgoda ułatwiona za pośrednictwem mrówek — U stołu wytacza się rzecz o łowach — Powieść Wojskiego o Rejtanie i księciu Denassów, przerwana — Zagajenie układów między stronami, także przerwane — Zjawisko z kluczem — Kłótnia — Hrabia z Gerwazym 

In [8]:
import json
import requests

stopwords_url = "https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt"
stopwords = set(requests.get(stopwords_url).text.splitlines())

def tokenize_and_clean(line):
    import re
    words = re.findall(r'\w+', line.lower()) 
    return [word for word in words if word not in stopwords]

words_rdd = rdd.flatMap(tokenize_and_clean)

word_counts = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

word_count_dict = word_counts.collectAsMap()

with open("pan_tadeusz_bag_of_words.json", "w", encoding="utf-8") as json_file:
    json.dump(word_count_dict, json_file, ensure_ascii=False, indent=4)

most_frequent_word = max(word_count_dict, key=word_count_dict.get)
print(f"Najczęściej występujące słowo (po odfiltrowaniu stop words): {most_frequent_word}")

Najczęściej występujące słowo (po odfiltrowaniu stop words): rzekł
